In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchsummary import summary
from transformers import BertModel, BertTokenizer


import pandas as pd

/home/henrytsui/anaconda3/envs/IDS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BERT_PATH = 'bert-base-uncased'
MAX_LEN = 64
DEVICE = "cuda"

In [4]:
bert = BertModel.from_pretrained(BERT_PATH)
tokenizer = BertTokenizer.from_pretrained(BERT_PATH)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
class Stock(Dataset):
    def __init__(self, df) -> None:
        self.pre = df["preprice"]
        self.post = df["latprice"]
        self.content = [tokenizer(text,padding='max_length', 
                       max_length = MAX_LEN, 
                       truncation=True,
                       return_tensors="pt") for text in df["content"]]

    def __getitem__(self, idx):
        return self.post[idx] - self.pre[idx], self.content[idx]

    def __len__(self):
        return merge.shape[0]

merge = pd.read_csv("../data/merge.csv", index_col=0)
stock = Stock(merge)
stock_loader = DataLoader(stock, batch_size=2)

In [6]:
num, con = next(iter(stock_loader))
print(con)

{'input_ids': tensor([[[  101, 10474,  2522,  1011,  3910,  2990, 27332, 14456,  2002,  3473,
           2194,  1005,  2205,  3435,  1005,  2004, 14163,  6711,  3913, 27475,
           2202,  3466,   102,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0]],

        [[  101,  1996,  2991,  1997,  2502,  6627,  2003, 12992,  2075,  4518,
          24110,  3215,  2006,  2813,  2395,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,

In [7]:
print(con["attention_mask"][0].shape)

torch.Size([1, 64])


In [8]:
bert = bert.to(DEVICE)

In [9]:
mask = con["attention_mask"].to(DEVICE)
text = con["input_ids"].squeeze(1).to(DEVICE)
print(mask, text)
# bert(con["attention_mask"])

tensor([[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]],
       device='cuda:0') tensor([[  101, 10474,  2522,  1011,  3910,  2990, 27332, 14456,  2002,  3473,
          2194,  1005,  2205,  3435,  1005,  2004, 14163,  6711,  3913, 27475,
          2202,  3466,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101

In [10]:
output = bert(text, mask)

In [12]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3090,  0.1375, -0.1950,  ..., -0.4220,  0.5317,  0.0663],
         [ 1.0220, -0.0362, -0.0199,  ...,  0.2628,  0.8525, -0.2002],
         [ 0.6710, -0.4314, -0.4480,  ..., -0.3828,  0.5428, -0.0963],
         ...,
         [ 0.2823,  0.0117, -0.1202,  ...,  0.1335,  0.2461, -0.0830],
         [-0.2138, -0.3979, -0.3595,  ...,  0.3255,  0.4415, -0.2490],
         [ 0.0946,  0.0277, -0.1058,  ...,  0.1156,  0.0821, -0.0057]],

        [[-0.0510,  0.1963,  0.2168,  ..., -0.4943,  0.3540,  0.3200],
         [-0.1961, -0.5545, -0.4311,  ..., -0.2283,  0.3729, -0.0934],
         [-0.9456, -0.6790,  0.2886,  ..., -0.7133,  0.6041, -0.5557],
         ...,
         [ 0.1780, -0.2627,  0.3362,  ...,  0.0462,  0.2970,  0.1298],
         [ 0.1960, -0.4050,  0.3419,  ..., -0.0775,  0.2562, -0.0417],
         [ 0.1667, -0.2292,  0.4816,  ..., -0.1900,  0.0820, -0.0516]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackw

In [ ]:
class Bert4price(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.bert = BertModel.from_pretrained(BERT_PATH)
        self.fc = nn.Linear(768, 5)